In [ ]:
import torch
torch.cuda.is_available()

In [ ]:

#from preprocessing import tokenize, export_to_csv
from gsdmm import MovieGroupProcess
import pickle
import pandas as pd
import numpy as np
%matplotlib inline


In [ ]:

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from IPython.display import clear_output
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)
import re

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
import spacy

In [ ]:
def stopword(text):
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def stemwords(text):    
    ps = PorterStemmer()
    text = text.lower().split()
    stemmed_words = [ps.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [ ]:
#This pickle file is obatined by filtering english language users who have atleast 20 tweets on their home timeline
#It is straightforward hence we generate a pickle file of filtered dataset and use here directly.

file = open("filtered_english.pkl",'rb')
df = pickle.load(file)
file.close()

In [ ]:

nlp = spacy.load('en_core_web_sm',parser=False, entity=False)
import time
start_time = time.time()

file = open("filtered_english.pkl",'rb')
df = pickle.load(file)
file.close()

df.drop(["engaging_user_id","tweet_id","tweet_timestamp","reply_timestamp","retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"], axis = 1)
df['text_tokens']= df['text_tokens'].str.split('\t').apply(lambda x:list(map(int, x)))
print ("--- %s seconds ---" % (time.time() - start_time))  

df['text_tokens'] = df['text_tokens'].apply(lambda x:tokenizer.convert_ids_to_tokens(x))
print ("--- %s seconds ---" % (time.time() - start_time))  


In [ ]:

def conv(s):
    l = []
    for i in range(len(s)):
        if s[i][:2]!='##':
            l.append(s[i])
        else:
            l[-1]=l[-1]+s[i][2:]
    return l


df['text_tokens'] = df['text_tokens'].apply(lambda x: conv(x))

In [ ]:
df['text_tokens'].head()

In [ ]:
df['text_tokens']= df['text_tokens'].apply(lambda x: " ".join([y for y in x]))
print ("--- %s seconds ---" % (time.time() - start_time))  

for r in (("[CLS]", ""),("[SEP]",""),("[UNK]",""),("RT","")):
    df['text_tokens'] = df['text_tokens'].apply(lambda x: x.replace(*r))
print ("--- %s seconds ---" % (time.time() - start_time))  

df["text_tokens"] = df["text_tokens"].apply(lambda x:' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))
print ("--- %s seconds ---" % (time.time() - start_time))  

df["text_tokens"] = df["text_tokens"].apply(lambda x: stemwords(x))
print ("--- %s seconds ---" % (time.time() - start_time))

df["text_tokens"] = df["text_tokens"].apply(lambda x: stopword(x))
print ("--- %s seconds ---" % (time.time() - start_time))  


#df.to_pickle('text_for_topics_1.pkl')
 


In [ ]:
df['text_tokens'].head()

In [ ]:
df['text_tokens'] = df['text_tokens'].apply(lambda x: x.split())

In [ ]:
docs = df['text_tokens'].tolist()
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)

print("Voc size:", n_terms)
print("Number of documents:", len(docs))

In [ ]:
mgp = MovieGroupProcess(K=50, alpha=0.1, beta=0.1, n_iters=20)

In [ ]:
# Train a new model 

# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm

start_time = time.time()
vocab = set(x for doc in docs for x in doc)
n_terms = len(vocab)
n_docs = len(docs)

# Fit the model on the data given the chosen seeds
y = mgp.fit(docs, n_terms)
print ("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topics :', sorted(doc_count))
print('*'*20)

# Topics sorted by document inside
top_index = doc_count.argsort()[-10:][::-1]
print('Most important clusters (by number of docs inside):', top_index)
print('*'*20)

In [ ]:
def top_words(distribution, top_index, num_words):
    for topic in top_index:
        pairs = sorted([(k, v) for k, v in distribution[topic].items()], key=lambda x: x[1], reverse=True)
        a = pairs[:num_words]
        c = [i[0] for i in a]
        print (c)
        print('.'*30)

b = top_words(mgp.cluster_word_distribution, top_index, 1000)        

In [ ]:
df['text_tokens'] = df['text_tokens'].apply(lambda x: mgp.choose_best_label(x)[0])

In [ ]:
df = df.reset_index(drop = True)

In [ ]:
with open('gsdmm_topics.pkl', "wb") as f:
     pickle.dump(df, f)
     f.close()